# Given any MILP make Benders' decomposition using provided hints

## Original problem
$$
\begin{align*}
\min_{x,y} \quad & f'y + c'x \\
\mathrm{s.t.} \quad & Ax + By [<=>] d \\
& l \leq x \leq u \\
& y \in \{0,1\}
\end{align*}
$$

## Restricted Master Problem
$$
\begin{align*}
\min_{y,z} \quad & z \\
\mathrm{s.t.} \quad & z \geq f'y + (d-By)'w^A_{i,k} + l_k'w^l_{i,k} - u'_k w^u_{i,k}, & i \in \mathrm{OptimalityCuts}, \ k=1,\dots,K \\
& (d_k-By)'w^A_{i,k} + l_k' w^l_{i,k} - u_k' w^u_{i,k} \leq 0, & i \in \mathrm{FeasibilityCuts}, \ k=1,\dots,K
\end{align*}
$$

## Subproblem(k)
$$
\begin{align*}
\min_{x} \quad & f'y^* + c'x  \\
\mathrm{s.t.} \quad & Ax [<=>] d - By^* \\
& l \leq x \leq u
\end{align*}
$$

### Dual of subproblem(k)
$$
\begin{align*}
\max_{w^A,w^l,w^u} \quad & (d-By^*)' w^A + l' w^l - u' w^u \\
\mathrm{s.t.} \quad & A' w^A + w^l - w^u = c \\
& w^A_i \in \mathbb{R}^m, \quad \forall i \in \{i:A_i x = d_i - B_i y^* \} \\
& w^A_i \geq 0 , \quad \forall i \in \{i:A_i x \geq d_i - B_i y^* \} \\
& w^A_i \leq 0, \quad \forall i \in \{i:A_i x \leq d_i - B_i y^* \} \\
& w^l,w^u \geq 0
\end{align*}
$$

In [1]:
from gurobipy import *
import pandas as pd
import numpy as np

UB = 1e3

supply = [10.,30.,40.,20]
demand = [20.,50.,30.]
varcost = [[2.,3.,4.], [3.,2.,1.], [1.,4.,3.], [4.,5.,2.]]
fixedcost = [[10.,30.,20.], [10.,30.,20.], [10.,30.,20.], [10.,30.,20.]]
ns = len(supply)
nd = len(demand)

xup = {(i,j):min(supply[i], demand[j]) for i in range(ns) for j in range(nd)}

## Original problem

In [2]:
milp = Model('milp')
x0 = {(i,j):milp.addVar(0, UB, 0., GRB.CONTINUOUS, 'x[%d,%d]' % (i,j)) for i in range(ns) for j in range(nd)}
y0 = {(i,j):milp.addVar(0, 1, 0., GRB.BINARY, 'y[%d,%d]'%(i,j)) for i in range(ns) for j in range(nd)}
milp.update()
cons_supply = [milp.addConstr( sum([x0[(i,j)] for j in range(nd)]) <= supply[i])  for i in range(ns)]
cons_demand = [milp.addConstr( sum([x0[(i,j)] for i in range(ns)]) >= demand[j]) for j in range(nd)]
cons_bigM   = [milp.addConstr(  x0[(i,j)] <= xup[(i,j)]*y0[(i,j)]) for i in range(ns) for j in range(nd)]
f = fixedcost
c = varcost
milp.setObjective(sum([f[i][j]*y0[(i,j)] + c[i][j]*x0[(i,j)] for i in range(ns) for j in range(nd)]),
                  GRB.MINIMIZE)

In [3]:
milp.Params.OutputFlag = 0
milp.optimize()

## Split original into RMP and Sub

### Restricted Master Problem
$$
\begin{align*}
\min_{y,z} \quad & z \\
\mathrm{s.t.} \quad & z \geq f'y + (d-By)'w^A_{i,k} + l_k'w^l_{i,k} - u'_k w^u_{i,k}, & i \in \mathrm{OptimalityCuts}, \ k=1,\dots,K \\
& (d_k-By)'w^A_{i,k} + l_k' w^l_{i,k} - u_k' w^u_{i,k} \leq 0, & i \in \mathrm{FeasibilityCuts}, \ k=1,\dots,K
\end{align*}
$$

### Original problem
$$
\begin{align*}
\min_{x,y} \quad & f'y + c'x \\
\mathrm{s.t.} \quad & Ax + By = d \\
& l \leq x \leq u \\
& y \in \{0,1\}
\end{align*}
$$

In [4]:
from scipy.sparse import coo_matrix
from builtins import range

def split_constraints(model):
    """
    Splits constraints into continuous and integer parts:
    Ax + By [<=>] d
    """
    constrs = model.getConstrs()    
    xs = [x for x in model.getVars() if x.VType == GRB.CONTINUOUS]
    x_inds = {x:i for i,x in enumerate(xs)}
    ys = [x for x in model.getVars() if x.VType in (GRB.INTEGER, GRB.BINARY)]
    y_inds = {x:i for i,x in enumerate(ys)}
    # Need to make index dictionary since __eq__ method of grb var 
    # prevents use of list(vars).index(...)    
    
    csenses = []
    d = []

    xdata = []
    xrow_inds = []
    xcol_inds = []

    ydata = []
    yrow_inds = []
    ycol_inds = []

    for row_idx, constr in enumerate(constrs):
        csenses.append(constr.Sense)        
        d.append(constr.RHS)
        row = model.getRow(constr)        
        for j in range(row.size()):            
            coeff = row.getCoeff(j)
            vj = row.getVar(j)
            if vj in x_inds:
                col_idx = x_inds[vj]
                xdata.append(coeff)
                xcol_inds.append(col_idx)
                xrow_inds.append(row_idx)
                # print 'Found A[%d,%d]=%g' % (row_idx, col_idx, coeff)                
            elif vj in y_inds:            
                col_idx = y_inds[vj]
                ydata.append(coeff)
                ycol_inds.append(col_idx)
                yrow_inds.append(row_idx)
                # print 'Found B[%d,%d]=%g' % (row_idx, col_idx, coeff)
            else:
                raise Exception('vj should be in either x_inds or y_inds')

    M  = len(constrs)    
    nx = len(xs)
    ny = len(ys)
    A = coo_matrix((xdata, (xrow_inds, xcol_inds)), shape=(M,nx)).tocsr()
    B = coo_matrix((ydata, (yrow_inds, ycol_inds)), shape=(M,ny)).tocsr()

    return A, B, d, csenses, xs, ys

In [5]:
### Subproblem
class Decomposer(object):
    def __init__(self, milp):
        self.milp = milp
        self._INF = 1e6
        self._master = None
        self._sub = None        
        self._A = None
        self._B = None
        self._d = None        
        self._csenses = None
        self._xs = None
        self._ys = None
        self._wa = None
        self._wl = None
        self._wu = None
        self._xl = None
        self._xu = None
        self._cx  = None
        self._fy  = None            
        
    def benders_decomp(self):
        """
        TODO: for multiple subproblems.
        """
        self._split_constraints()
        
        master = self.make_master()
        sub = self.make_sub()
        self._master = master
        self._sub = sub
        
        return master, sub
    
    def _split_constraints(self):
        A,B,d,csenses,xs,ys = split_constraints(self.milp)
        self._A = A
        self._B = B
        self._d = d
        self._csenses = csenses
        self._x0 = xs
        self._y0 = ys
        
    def make_master(self):
        LB = -self._INF
        UB = self._INF
        if self._y0 is None:
            self._split_constraints()
            
        ys0 = self._y0
        ny = len(ys0)        
        B = self._B
        fy = [yj.Obj for yj in ys0]
        master = Model('master')
        z = master.addVar(LB, UB, 0., GRB.CONTINUOUS, 'z')
        ys = [master.addVar(y.LB, y.UB, y.Obj, y.VType, y.VarName) for y in ys0]
        master.addConstr(z >= sum([fy[j]*ys[j] for j in range(ny)]))
        master.setObjective(z, GRB.MINIMIZE)
                
        self._z = z
        self._ys = ys
        self._fy = fy
        
        return master
        
    def make_sub(self, yopt=None):
        """
        Constraint doesn't change.
        Objective changes with RMP solution, so yopt is optional when initiating.
        """
        LB = -self._INF
        UB = self._INF
        
        if self._x0 is None:
            self._split_constraints()
        
        xs0 = self._x0
        A = self._A
        d = self._d
        csenses = self._csenses        

        m = len(d)
        n = len(xs0)
        nx = n
        sub = Model('sub')
        lb_dict = {GRB.EQUAL: -self._INF, GRB.GREATER_EQUAL: 0., GRB.LESS_EQUAL: -self._INF}
        ub_dict = {GRB.EQUAL: self._INF, GRB.GREATER_EQUAL: self._INF, GRB.LESS_EQUAL: 0.}
        wa = [sub.addVar(lb_dict[sense], ub_dict[sense], 0., GRB.CONTINUOUS, 'wa[%d]'%i) for i,sense in enumerate(csenses)]
        wl = [sub.addVar(0., UB, 0., GRB.CONTINUOUS, 'wl[%d]'%i) for i in range(n)]
        wu = [sub.addVar(0., UB, 0., GRB.CONTINUOUS, 'wu[%d]'%i) for i in range(n)]
        xl = [x.LB for x in xs0]
        xu = [x.UB for x in xs0]
        cx  = [x.Obj for x in xs0]
        
        self._xl = xl
        self._xu = xu
        self._cx  = cx
        self._wa = wa
        self._wl = wl
        self._wu = wu
            
        # This dual constraint never changes
        dual_cons = [sub.addConstr(sum([A[i,j]*wa[i] for i in range(m)]) + \
                                   wl[j] - wu[j] == cx[j], name=xs0[j].VarName) for j in range(nx)]
        
        if yopt is not None:
            sub.setObjective(sum([d[i]*wa[i]-sum([B[i,j]*yopt[j]*wa[i] for j in range(ny)]) for i in range(m)]) + \
                             sum([xl[j]*wl[j] for j in range(n)]) - \
                             sum([xu[j]*wu[j] for j in range(n)]), GRB.MAXIMIZE)
        
        return sub

    def update_subobj(self, yopt):
        sub = self._sub
        if sub is None:
            sub = self.make_sub(yopt)
        
        d = self._d
        B = self._B
        wa = self._wa
        wl = self._wl
        wu = self._wu
        xl = self._xl
        xu = self._xu
        
        m,ny = B.shape
        n = len(xl)
        
        sub.setObjective(sum([d[i]*wa[i]-sum([B[i,j]*yopt[j]*wa[i] for j in range(ny)]) for i in range(m)]) + \
                         sum([xl[j]*wl[j] for j in range(n)]) - \
                         sum([xu[j]*wu[j] for j in range(n)]), GRB.MAXIMIZE)

    def make_optcut(self):        
        z = self._z
        fy = self._fy
        ys = self._ys
        d = self._d
        B = self._B
        xl = self._xl
        xu = self._xu        
        m = len(d)
        n = len(xl)
        ny = len(ys)
        
        wa = [w.X for w in self._wa]
        wl = [w.X for w in self._wl]
        wu = [w.X for w in self._wu]                
        
        cut = z >= sum([fy[j]*ys[j] for j in range(ny)]) + \
            sum([d[i]*wa[i]-sum([B[i,j]*ys[j]*wa[i] for j in range(ny)]) for i in range(m)]) + \
            sum([xl[j]*wl[j] for j in range(n)]) - \
            sum([xu[j]*wu[j] for j in range(n)])

        return cut

    def make_feascut(self):
        """
        THIS MAY NOT WORK if dvar.X doesn't work for infeas problems.
        Instead, may need FarkasDuals for primal sub problem
        """
        # Get unbounded ray
        wa = [w.X for w in self._wa]
        wl = [w.X for w in self._wl]
        wu = [w.X for w in self._wu]

        ys = self._ys
        d = self._d
        B = self._B
        xl = self._xl
        xu = self._xu
        wa = self._wa
        wl = self._wl
        wu = self._wu
        m = len(d)
        n = len(xl)

        cut = sum([d[i]*wa[i]-sum([B[i,j]*ys[j]*wa[i] for j in range(ny)]) for i in range(m)]) + \
                             sum([xl[j]*wl[j] for j in range(n)]) - \
                             sum([xu[j]*wu[j] for j in range(n)]) <= 0

        return cut
    
    def calc_sub_objval(self, yopt):
        sub = self._sub
        fy = self._fy
        #objval = sub.ObjVal + sum([fj*yj for fj,yj in zip(fy,yopt)])
        objval = sub.ObjVal + sum([fy[j]*yopt[j] for j in range(len(yopt))])
        
        return objval
    
    def get_sub(self):
        if self._sub is None:
            self._sub = self.make_sub()

        return self._sub

    def get_master(self):
        if self._master is None:
            self._master = self.make_master()
            
        return self._master

In [6]:
###==================================================###
### SOLVER-SPECIFIC CALLBACK
### Feas and Opt cuts implemented as lazy constraints
###==================================================###
from six import iteritems

def benders(model, where):
    GAPTOL = 1e-6
    
    if where in (GRB.Callback.MIPSOL, GRB.Callback.MIPNODE):
        ### Lazy constraints only allowed for MIPNODE or MIPSOL
        ys = decomposer._ys
        z  = decomposer._z        
        if where==GRB.Callback.MIPSOL:
            yopt = [model.cbGetSolution(y) for y in ys]
            #yopt = {j:model.cbGetSolution(y) for j,y in iteritems(ys)}
            zmaster = model.cbGetSolution(z)
        elif where==GRB.Callback.MIPNODE:        
            yopt = [model.cbGetNodeRel(y) for y in ys]
            #yopt = {j:model.cbGetNodeRel(y) for j,y in iteritems(ys)}
            zmaster = model.cbGetNodeRel(z)
        #************
        print('#'*40)
        print('Updating subproblem objective')
        print('#'*40)
        #************
        decomposer.update_subobj(yopt)        
        sub = decomposer.get_sub()        
        sub.optimize()
        
        if sub.Status == GRB.Status.UNBOUNDED:
            print('#'*40)
            print('Adding feasibility cut')
            print('#'*40)
            # Add feasibility cut, ensuring that cut indeed eliminates current incumbent
            feascut = decomposer.make_feascut(yopt, zmaster)
            model.cbLazy(feascut)            
        else:
            print('#'*40)
            print('Adding optimality cut')
            print('#'*40)
            ### Otherwise add Optimality cut                        
            #try:                
            zsub = decomposer.calc_sub_objval(yopt)
            #except:
            #zsub = -1e6
            gap = zmaster - zsub
            print('*'*40)    
            print('zSub: %g' % zsub)
            print('zMaster: %g' % zmaster)                        
            print('Gap: %g' % gap)
            print('*'*40)

            if abs(gap) > GAPTOL:
                optcut = decomposer.make_optcut()
                model.cbLazy(optcut)                
            else:
                # Accept as new incumbent
                pass

In [7]:
decomposer = Decomposer(milp)

In [8]:
master,sub = decomposer.benders_decomp()
sub.Params.OutputFlag=0
master.Params.LazyConstraints = 1
master.Params.OutputFlag=0
master.optimize(benders)

Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
########################################
Updating subproblem objective
########################################
########################################
Adding optimality cut
########################################
****************************************
zSub: 1e+08
zMaster: 0
Gap: -1e+08
****************************************
########################################
Updating subproblem objective
########################################
########################################
Adding optimality cut
########################################
****************************************
zSub: 1e+08
zMaster: 0
Gap: -1e+08
****************************************
########################################
Updating subproblem objective
########################################
########################################
Adding optimality cut
########################################
*******************************

In [9]:
print('Master objval=%g. Original objval=%g. Diff=%g' % (master.ObjVal, milp.ObjVal, master.ObjVal-milp.ObjVal))

Master objval=350. Original objval=350. Diff=-4.00001e-05


In [10]:
### Resolve sub with final yopt to get x values or solve primal
ys2 = [y for y in master.getVars() if y.VType != 'C']
yfin = [y.X for y in ys2]
decomposer.update_subobj(yfin)
sub.Params.OutputFlag=0
sub.optimize()
dual_cons = sub.getConstrs()
xs2 = {r.ConstrName:r.Pi for r in dual_cons}

In [22]:
xs1 = [x for x in milp.getVars() if x.VType=='C']
print('%15.10s%15.10s%15.10s' % ('x0','x_benders','Diff'))
for xj1 in xs1:
    xj2 = xs2[xj1.VarName]
    print('%15.3g%15.3g%15.3g' % (xj1.X, xj2, xj2-xj1.X))

             x0      x_benders           Diff
              0              0              0
              0              0              0
             10             10         -4e-05
              0              0              0
             30             30              0
              0              0              0
             20             20              0
             20             20              0
              0              0              0
              0              0              0
              0              0              0
             20             20              0


In [21]:
ys1 = [y for y in milp.getVars() if y.VType!='C']
print('%15.10s%15.10s%15.10s' % ('y0','y_RMP','Diff'))
for yj1 in ys1:
    yj2 = master.getVarByName(yj1.VarName)
    print('%15.3g%15.3g%15.3g' % (yj1.X, yj2.X, yj2.X-yj1.X))

             y0          y_RMP           Diff
              0          4e-06          4e-06
             -0       3.77e-15       3.77e-15
              1              1         -4e-06
             -0          4e-12          4e-12
              1              1              0
             -0             -0              0
              1              1              0
              1              1              0
              0             -0             -0
             -0       1.11e-16       1.11e-16
             -0             -0              0
              1              1              0


In [13]:
### Double-check using primal of subproblem
Ap, Bp, dp, psenses, x0p, y0p = split_constraints(milp)
mA,nA = Ap.shape
psub = Model('psub')
xps = [psub.addVar(x0i.LB, x0i.UB, 0., GRB.CONTINUOUS, x0i.VarName) for x0i in x0p]
cp  = [x0i.Obj for x0i in x0p]
psub.setObjective(sum([cp[j]*xps[j] for j in range(len(xps))]), GRB.MINIMIZE)
yopt0 = [y.X for y in milp.getVars() if y.VType!='C']
#pconstrs = []
for i,csense in enumerate(psenses):    
    expr_L = sum([Ap[i,j]*xps[j] for j in range(len(xps))])
    expr_R = dp[i] - sum([Bp[i,j]*yopt0[j] for j in range(len(yopt0))])
    psub.addConstr(expr_L, csense, expr_R)
psub.optimize()

Optimize a model with 19 rows, 12 columns and 36 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+03, 1e+03]
  RHS range        [1e+01, 5e+01]
Presolve removed 19 rows and 12 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.01 seconds
Optimal objective  2.400000000e+02


In [14]:
print("f'y=%g"%sum([y.Obj*y.X for y in y0p]))
print("c'x=%g"%sum([x.Obj*x.X for x in x0p]))

f'y=110
c'x=240


In [15]:
print('Subproblem:')
print("c'x=%g" % sum([x.Obj*x.X for x in xps]))

Subproblem:
c'x=240


In [16]:
print('%15.10s%15.10s%15.10s%15.10s%15.10s' % ('x0','x_sub','Diff', 'c0', 'c_sub'))
for xj0 in x0p:
    xj2 = psub.getVarByName(xj0.VarName)
    print('%15.10g%15.10g%15.10g%15.10g%15.10g' % (xj0.X, xj2.X, xj2.X-xj0.X, xj0.Obj, xj2.Obj))

             x0          x_sub           Diff             c0          c_sub
              0              0              0              2              2
              0              0              0              3              3
             10             10              0              4              4
              0              0              0              3              3
             30             30              0              2              2
              0              0              0              1              1
             20             20              0              1              1
             20             20              0              4              4
              0              0              0              3              3
              0              0              0              4              4
              0              0              0              5              5
             20             20              0              2              2


In [17]:
### Double-check using dual of subproblem
INF = 1e6
Ap, Bp, dp, psenses, x0p, y0p = split_constraints(milp)
mA,nA = Ap.shape
dsub = Model('dsub')
wa_lbs = {GRB.EQUAL: -INF, GRB.GREATER_EQUAL:0., GRB.LESS_EQUAL:-INF}
wa_ubs = {GRB.EQUAL: INF,  GRB.GREATER_EQUAL:INF,GRB.LESS_EQUAL:0.}
wa = [dsub.addVar(wa_lbs[sense], wa_ubs[sense], 0., GRB.CONTINUOUS, 'wa[%d]'%i) for i,sense in enumerate(psenses)]
wl = [dsub.addVar(0., INF, 0., GRB.CONTINUOUS, 'wl[%d]'%j) for j in range(nA)]
wu = [dsub.addVar(0., INF, 0., GRB.CONTINUOUS, 'wu[%d]'%j) for j in range(nA)]
cp  = [x0i.Obj for x0i in x0p]
xl = [x.LB for x in x0p]
xu = [x.UB for x in x0p]
yopt0 = [y.X for y in milp.getVars() if y.VType!='C']
ny = len(yopt0)

dsub.setObjective(sum([dp[i]*wa[i]-sum([Bp[i,j]*yopt0[j]*wa[i] for j in range(ny)])  for i in range(mA)]) + \
                  sum([xl[j]*wl[j] for j in range(nA)]) - \
                  sum([xu[j]*wu[j] for j in range(nA)]),
                  GRB.MAXIMIZE)

ddual_cons = [dsub.addConstr(sum([Ap[i,j]*wa[i] for i in range(mA)]) + wl[j] - wu[j] == cp[j], name=x0p[j].VarName) for j in range(nA)]

dsub.optimize()

Optimize a model with 12 rows, 43 columns and 60 nonzeros
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+01, 1e+03]
  Bounds range     [1e+06, 1e+06]
  RHS range        [1e+00, 5e+00]
Presolve removed 0 rows and 19 columns
Presolve time: 0.00s
Presolved: 12 rows, 24 columns, 41 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5000041e+08   1.200003e+07   0.000000e+00      0s
      12    2.4000000e+02   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.01 seconds
Optimal objective  2.400000000e+02


In [18]:
xd = [r.Pi for r in ddual_cons]

print('%15.10s%15.10s%15.10s%15.10s%15.10s' % ('x0','x_sub','Diff', 'c0', 'c_sub'))
for j,xj0 in enumerate(x0p):
    xj2 = xd[j]
    print('%15.10g%15.10g%15.10g%15.10g%15.10g' % (xj0.X, xj2, xj2-xj0.X, xj0.Obj, cp[j]))

             x0          x_sub           Diff             c0          c_sub
              0             -0             -0              2              2
              0             -0             -0              3              3
             10             10              0              4              4
              0             -0             -0              3              3
             30             30              0              2              2
              0             -0             -0              1              1
             20             20              0              1              1
             20             20              0              4              4
              0             -0             -0              3              3
              0             -0             -0              4              4
              0             -0             -0              5              5
             20             20              0              2              2


## Subproblem(k)
$$
\begin{align*}
\min_{x} \quad & f'y^* + c'x  \\
\mathrm{s.t.} \quad & Ax [<=>] d - By^* \\
& l \leq x \leq u
\end{align*}
$$

### Restricted Master Problem
$$
\begin{align*}
\min_{y,z} \quad & z \\
\mathrm{s.t.} \quad & z \geq f'y + (d-By)'w^A_{i,k} + l_k'w^l_{i,k} - u'_k w^u_{i,k}, & i \in \mathrm{OptimalityCuts}, \ k=1,\dots,K \\
& (d_k-By)'w^A_{i,k} + l_k' w^l_{i,k} - u_k' w^u_{i,k} \leq 0, & i \in \mathrm{FeasibilityCuts}, \ k=1,\dots,K
\end{align*}
$$

### Dual of subproblem(k)
$$
\begin{align*}
\max_{w^A,w^l,w^u} \quad & (d-By^*)' w^A + l' w^l - u' w^u \\
\mathrm{s.t.} \quad & A' w^A + w^l - w^u = c \\
& w^A \in \mathbb{R}^m \\
& w^l,w^u \geq 0
\end{align*}
$$